In [3]:
import pandas as pd 
import numpy as np
from langdetect import detect
import seaborn as sns
import io 
#from s3_credentials import *

In [5]:
import boto3

In [11]:
# boto session 
YOUR_ACCESS_KEY = 
YOUR_SECRET_KEY = 

session = boto3.Session(aws_access_key_id= YOUR_ACCESS_KEY, 
                        aws_secret_access_key= YOUR_SECRET_KEY)

s3 = session.resource("s3")
client = session.client("s3")

### Pull datasets

In [52]:
#Pull subdatasets

# real
obj = s3.Object('jedha-fake-reviews-project', "datasets/real_reviews_raw.csv")
real = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), low_memory = False, index_col=0)

# fake
obj = s3.Object('jedha-fake-reviews-project', "datasets/fake_reviews_raw.csv")
fake = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), low_memory = False, index_col=0)


### Merge both datasets

In [56]:
# find order columned list
columns_list_ordered = real.columns.values.tolist()
columns_list_ordered.append('url_not_recommended')

# create mapping table 
pivot_table = real.loc[:, ['url','restaurant_average_rating','restaurant_reviews_count','restaurant_expensiveness','restaurant_name']].drop_duplicates('url')

# merge fake with mapping table
fake_merged = pd.merge(fake, pivot_table, left_on='url', right_on = 'url', how='left')

fake_dataset_reworked = fake_merged.loc[:, columns_list_ordered]
# second method --> fake_dataset_reworked = fake_merged.reindex(columns = columns_list_ordered) 
# this method requires to add suffixes=("_Drop", None) to previosu merge


# concat both and put into full_dataset
full_dataset = pd.concat([real, fake_dataset_reworked])
full_dataset.loc[(full_dataset['restaurant_expensiveness'] == '14' ) | (full_dataset['restaurant_expensiveness'] == '10' ), 'restaurant_expensiveness'] = 'N/C'
full_dataset = full_dataset.drop_duplicates()
full_dataset = full_dataset.reset_index(drop = True)
full_dataset['url_not_recommended'] = full_dataset['url'].apply(lambda url: url.replace("biz/", "not_recommended_reviews/"))

### (Optional) Function to return language for each 

In [ ]:
from langdetect import detect
# def function 
def get_lang(x):

    try:
        lang = detect(str(x))
    except:
        lang = 'not_found'

    return lang

In [ ]:
# apply to df 
full_dataset['language'] = full_dataset["text_review"].apply(lambda x : get_lang(x))

### Push to bucket

In [ ]:
# upload new dataset 
# set path and bucket name
PATH = "datasets/full_dataset.csv"
bucket = s3.Bucket(name = "jedha-fake-reviews-project")
# export dataset as csv
data = full_dataset.to_csv()
put_object = bucket.put_object(ACL='private', Key= PATH, Body=data)
#check 
for obj in bucket.objects.all():
    print(obj.key)